In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.metrics import r2_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_col

In [ ]:
df1 = pd.read_excel('data_word_dummies.xlsx')

In [ ]:
df2 = pd.read_excel('data_themes.xlsx')

In [ ]:
df3 = pd.read_excel('data_pca.xlsx')

In [ ]:
df1['log_est'] = np.log1p(df1['est_max'])
df2['log_est'] = np.log1p(df2['est_max'])
df3['log_est'] = np.log1p(df3['est_max'])

In [ ]:
text_features = [col for col in df1.columns if col.startswith('has_')]
X_struct = df1[['log_est', 'year', 'auction_house', 'online']]
y = df1['log_realised_rmb']

In [ ]:
text_features = [col for col in text_features if col not in ['has_钤印', 'has_印文']]

In [ ]:
df_model = df1.dropna(subset=['log_realised_rmb', 'log_est', 'auction_house', 'year', 'online', 'desc_len'])

In [ ]:
# Target
y = df_model['log_realised_rmb']

# Model 1
X1 = df_model[['log_est']]
X1 = sm.add_constant(X1)
model1 = sm.OLS(y, X1).fit()

# Model 2 
X2 = df_model[['log_est', 'year', 'auction_house_dummy', 'online']]
X2 = sm.add_constant(X2)
model2 = sm.OLS(y, X2).fit()

# Model 3 
X3 = df_model[['year', 'auction_house_dummy', 'online', 'desc_len']]
X3 = sm.add_constant(X3)
model3 = sm.OLS(y, X3).fit()

# Model 4
X4 = df_model[['year', 'auction_house_dummy', 'online', 'desc_len'] + text_features]
X4 = sm.add_constant(X4)
model4 = sm.OLS(y, X4).fit()

# Model 5
X5 = df_model[text_features]
X5 = sm.add_constant(X5)
model5 = sm.OLS(y, X5).fit()

# Model 6
X6 = df_model[['log_est', 'year', 'auction_house_dummy', 'online', 'desc_len'] + text_features]
X6 = sm.add_constant(X6)
model6 = sm.OLS(y, X6).fit()

In [ ]:
df_model_pca = df3.dropna(subset=['log_realised_rmb', 'log_est', 'auction_house', 'year', 'online', 'desc_len'])

In [ ]:
pca_features = [col for col in df_model_pca.columns if col.startswith('pca_')]
y = df_model_pca['log_realised_rmb']

In [ ]:
# Model 7
X7 = df_model_pca[pca_features]
X7 = sm.add_constant(X7)
model7 = sm.OLS(y, X7).fit()

# Model 8
X8 = df_model_pca[['year', 'auction_house_dummy', 'online', 'desc_len'] + pca_features]
X8 = sm.add_constant(X8)
model8 = sm.OLS(y, X8).fit()

In [ ]:
df_model_themed = df2.dropna(subset=['log_realised_rmb', 'log_est', 'auction_house', 'year', 'online', 'desc_len'])

In [ ]:
theme_features = [col for col in df_model_themed.columns if col.startswith('has_')]
y = df_model_themed['log_realised_rmb']

In [ ]:
# Model 9
X9 = df_model_themed[theme_features]
X9 = sm.add_constant(X9)
model9 = sm.OLS(y, X9).fit()

# Model 10
X10 = df_model_themed[['year', 'auction_house_dummy', 'online', 'desc_len'] + theme_features]
X10 = sm.add_constant(X10)
model10 = sm.OLS(y, X10).fit()

In [ ]:
staged_table = summary_col(
    results=[model1, model2, model3,model4, model5, model6, model7, model8, model9, model10],
    model_names=['m1', 'm2', 'm3', 'm4', 'm5', 'm6', 'm7', 'm8','Model 9', 'Model 10'],
    stars=True,
    float_format='%0.3f',
    info_dict={
        'R-squared': lambda x: f"{x.rsquared:.3f}",
        'N': lambda x: f"{int(x.nobs)}",
        'df_model': lambda x: f"{x.df_model:.0f}",
        'df_resid': lambda x: f"{x.df_resid:.0f}"
    }
)
print(staged_table)

LASSO

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV
from sklearn.metrics import r2_score

In [ ]:
lasso_coefs = {}

In [ ]:
lasso_r2 = {}

In [ ]:
# Model 1
X1 = df_model[['log_est']]
y = df_model['log_realised_rmb']

scaler1 = StandardScaler()
X1_scaled = scaler1.fit_transform(X1)

lasso1 = LassoCV(cv=5, random_state=42).fit(X1_scaled, y)

optimal_alpha1 = lasso1.alpha_
y_pred1 = lasso1.predict(X1_scaled)
r2_1 = r2_score(y, y_pred1)

print(f"Model 1: Optimal alpha = {optimal_alpha1:.4f}, R² = {r2_1:.3f}")

coefs1 = pd.Series(lasso1.coef_, index=X1.columns)
lasso_coefs['Model 1'] = coefs1
lasso_r2['Model 1'] = r2_1

In [ ]:
# Model 2
X2 = df_model[['log_est', 'year', 'auction_house_dummy', 'online']]
y = df_model['log_realised_rmb']

scaler2 = StandardScaler()
X2_scaled = scaler2.fit_transform(X2)

lasso2 = LassoCV(cv=5, random_state=42).fit(X2_scaled, y)

optimal_alpha2 = lasso2.alpha_
y_pred2 = lasso2.predict(X2_scaled)
r2_2 = r2_score(y, y_pred2)

print(f"Model 2: Optimal alpha = {optimal_alpha2:.4f}, R² = {r2_2:.3f}")

coefs2 = pd.Series(lasso2.coef_, index=X2.columns)
lasso_coefs['Model 2'] = coefs2
lasso_r2['Model 2'] = r2_2

In [ ]:
# Model 3
X3 = df_model[['year', 'auction_house_dummy', 'online', 'desc_len']]
y = df_model['log_realised_rmb']

scaler3 = StandardScaler()
X3_scaled = scaler3.fit_transform(X3)

lasso3 = LassoCV(cv=5, random_state=42).fit(X3_scaled, y)

optimal_alpha3 = lasso3.alpha_
y_pred3 = lasso3.predict(X3_scaled)
r2_3 = r2_score(y, y_pred3)

print(f"Model 3: Optimal alpha = {optimal_alpha3:.4f}, R² = {r2_3:.3f}")

coefs3 = pd.Series(lasso3.coef_, index=X3.columns)
lasso_coefs['Model 3'] = coefs3
lasso_r2['Model 3'] = r2_3

In [ ]:
# Model 4
X4 = df_model[['year', 'auction_house_dummy', 'online', 'desc_len'] + text_features]
y = df_model['log_realised_rmb']

scaler4 = StandardScaler()
X4_scaled = scaler4.fit_transform(X4)

lasso4 = LassoCV(cv=5, random_state=42).fit(X4_scaled, y)

optimal_alpha4 = lasso4.alpha_
y_pred4 = lasso4.predict(X4_scaled)
r2_4 = r2_score(y, y_pred4)

print(f"Model 4: Optimal alpha = {optimal_alpha4:.4f}, R² = {r2_4:.3f}")

coefs4 = pd.Series(lasso4.coef_, index=X4.columns)
lasso_coefs['Model 3'] = coefs4
lasso_r2['Model 4'] = r2_4

In [ ]:
# Model 5
X5 = df_model[text_features]
y = df_model['log_realised_rmb']

scaler5 = StandardScaler()
X5_scaled = scaler5.fit_transform(X5)

lasso5 = LassoCV(cv=5, random_state=42).fit(X5_scaled, y)

optimal_alpha5 = lasso5.alpha_
y_pred5 = lasso5.predict(X5_scaled)
r2_5 = r2_score(y, y_pred5)

print(f"Model 5: Optimal alpha = {optimal_alpha5:.4f}, R² = {r2_5:.3f}")

coefs5 = pd.Series(lasso5.coef_, index=X5.columns)
lasso_coefs['Model 5'] = coefs5
lasso_r2['Model 5'] = r2_5

In [ ]:
# Model 6
X6 = df_model[['log_est', 'year', 'auction_house_dummy', 'online', 'desc_len'] + text_features]
y = df_model['log_realised_rmb']

scaler6 = StandardScaler()
X6_scaled = scaler6.fit_transform(X6)

lasso6 = LassoCV(cv=5, random_state=42).fit(X6_scaled, y)

optimal_alpha6 = lasso6.alpha_
y_pred6 = lasso6.predict(X6_scaled)
r2_6 = r2_score(y, y_pred6)

print(f"Model 6: Optimal alpha = {optimal_alpha6:.4f}, R² = {r2_6:.3f}")

coefs6 = pd.Series(lasso6.coef_, index=X6.columns)
lasso_coefs['Model 6'] = coefs6
lasso_r2['Model 6'] = r2_6

In [ ]:
# Model 7
X7 = df_model_pca[pca_features]
y = df_model_pca['log_realised_rmb']

scaler7 = StandardScaler()
X7_scaled = scaler7.fit_transform(X7)

lasso7 = LassoCV(cv=5, random_state=42).fit(X7_scaled, y)

optimal_alpha7 = lasso7.alpha_
y_pred7 = lasso7.predict(X7_scaled)
r2_7 = r2_score(y, y_pred7)

print(f"Model 7: Optimal alpha = {optimal_alpha7:.4f}, R² = {r2_7:.3f}")

coefs7 = pd.Series(lasso7.coef_, index=X7.columns)
lasso_coefs['Model 7'] = coefs7
lasso_r2['Model 7'] = r2_7

In [ ]:
# Model 8
X8 = df_model_pca[['year', 'auction_house_dummy', 'online', 'desc_len'] + pca_features]
y = df_model_pca['log_realised_rmb']

scaler8 = StandardScaler()
X8_scaled = scaler8.fit_transform(X8)

lasso8 = LassoCV(cv=5, random_state=42).fit(X8_scaled, y)

optimal_alpha8 = lasso8.alpha_
y_pred8 = lasso8.predict(X8_scaled)
r2_8 = r2_score(y, y_pred8)

print(f"Model 8: Optimal alpha = {optimal_alpha8:.4f}, R² = {r2_8:.3f}")

coefs8 = pd.Series(lasso8.coef_, index=X8.columns)
lasso_coefs['Model 8'] = coefs8
lasso_r2['Model 8'] = r2_8


In [ ]:
# Model 9
X9 = df_model_themed[['year', 'auction_house_dummy', 'online', 'desc_len'] + theme_features]
y = df_model_themed['log_realised_rmb']

scaler9 = StandardScaler()
X9_scaled = scaler9.fit_transform(X10)

lasso9 = LassoCV(cv=5, random_state=42).fit(X9_scaled, y)

optimal_alpha9 = lasso9.alpha_
y_pred9 = lasso9.predict(X9_scaled)
r2_9 = r2_score(y, y_pred9)

print(f"Model 9: Optimal alpha = {optimal_alpha9:.4f}, R² = {r2_9:.3f}")

coefs10 = pd.Series(lasso10.coef_, index=X10.columns)
lasso_coefs['Model 9'] = coefs10
lasso_r2['Model 9'] = r2_10

In [ ]:
# Model 10
X10 = df_model_themed[theme_features]
y = df_model_themed['log_realised_rmb']

scaler10 = StandardScaler()
X10_scaled = scaler10.fit_transform(X11)

lasso10 = LassoCV(cv=5, random_state=42).fit(X11_scaled, y)

optimal_alpha10 = lasso11.alpha_
y_pred10 = lasso11.predict(X10_scaled)
r2_10 = r2_score(y, y_pred10)

print(f"Model 11: Optimal alpha = {optimal_alpha10:.4f}, R² = {r2_10:.3f}")

coefs10 = pd.Series(lasso11.coef_, index=X10.columns)
lasso_coefs['Model 10'] = coefs10
lasso_r2['Model 10'] = r2_10

In [ ]:
lasso_df = pd.DataFrame(lasso_coefs)
lasso_df = lasso_df.round(3)
r2_series = pd.Series(lasso_r2, name='R²')
lasso_df_with_r2 = pd.concat([r2_series.to_frame().T, lasso_df])

In [ ]:
lasso_df_with_r2

In [ ]:
gboost_importances = {}
gboost_r2 = {}

In [ ]:
# Model 1
X1 = df_model[['log_est']]
y = df_model['log_realised_rmb']

scaler1 = StandardScaler()
X1_scaled = scaler1.fit_transform(X1)

gboost1 = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gboost1.fit(X1_scaled, y)

y_pred1 = gboost1.predict(X1_scaled)
r2_1 = r2_score(y, y_pred1)

print(f"Gradient Boosting Model 1: R² = {r2_1:.3f}")

importances1 = pd.Series(gboost1.feature_importances_, index=X1.columns)
gboost_importances['Model 1'] = importances1
gboost_r2['Model 1'] = r2_1

In [ ]:
# Model 2
X2 = df_model[['log_est', 'year', 'auction_house_dummy', 'online']]
y = df_model['log_realised_rmb']

scaler2 = StandardScaler()
X2_scaled = scaler2.fit_transform(X2)

gboost2 = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gboost2.fit(X2_scaled, y)

y_pred2 = gboost2.predict(X2_scaled)
r2_2 = r2_score(y, y_pred2)

print(f"Gradient Boosting Model 2: R² = {r2_2:.3f}")

importances2 = pd.Series(gboost2.feature_importances_, index=X2.columns)
gboost_importances['Model 2'] = importances2
gboost_r2['Model 2'] = r2_2

In [ ]:
# Model 3
X3 = df_model[['year', 'auction_house_dummy', 'online', 'desc_len']]
y = df_model['log_realised_rmb']

scaler3 = StandardScaler()
X3_scaled = scaler3.fit_transform(X3)

gboost3 = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gboost3.fit(X3_scaled, y)

y_pred3 = gboost3.predict(X3_scaled)
r2_3 = r2_score(y, y_pred3)

print(f"Gradient Boosting Model 3: R² = {r2_3:.3f}")

importances3 = pd.Series(gboost3.feature_importances_, index=X3.columns)
gboost_importances['Model 3'] = importances3
gboost_r2['Model 3'] = r2_3

In [ ]:
# Model 4
X4 = df_model[['year', 'auction_house_dummy', 'online', 'desc_len'] + text_features]
y = df_model['log_realised_rmb']

scaler4 = StandardScaler()
X4_scaled = scaler4.fit_transform(X4)

gboost4 = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gboost4.fit(X4_scaled, y)

y_pred4 = gboost4.predict(X4_scaled)
r2_4 = r2_score(y, y_pred4)

print(f"Gradient Boosting Model 4: R² = {r2_4:.3f}")

importances4 = pd.Series(gboost4.feature_importances_, index=X4.columns)
gboost_importances['Model 4'] = importances4
gboost_r2['Model 4'] = r2_4

In [ ]:
# Model 5
X5 = df_model[text_features]
y = df_model['log_realised_rmb']

scaler5 = StandardScaler()
X5_scaled = scaler5.fit_transform(X5)

gboost5 = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gboost5.fit(X5_scaled, y)

y_pred5 = gboost5.predict(X5_scaled)
r2_5 = r2_score(y, y_pred5)

print(f"Gradient Boosting Model 5: R² = {r2_5:.3f}")

importances5 = pd.Series(gboost5.feature_importances_, index=X5.columns)
gboost_importances['Model 5'] = importances5
gboost_r2['Model 5'] = r2_5

In [ ]:
# Model 6
X6 = df_model[['log_est', 'year', 'auction_house_dummy', 'online', 'desc_len'] + text_features]
y = df_model['log_realised_rmb']

scaler6 = StandardScaler()
X6_scaled = scaler6.fit_transform(X6)

gboost6 = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gboost6.fit(X6_scaled, y)

y_pred6 = gboost6.predict(X6_scaled)
r2_6 = r2_score(y, y_pred6)

print(f"Gradient Boosting Model 6: R² = {r2_6:.3f}")

importances6 = pd.Series(gboost6.feature_importances_, index=X6.columns)
gboost_importances['Model 6'] = importances6
gboost_r2['Model 6'] = r2_6


In [ ]:
# Model 7
X7 = df_model_pca[pca_features]
y = df_model_pca['log_realised_rmb']

scaler7 = StandardScaler()
X7_scaled = scaler7.fit_transform(X6)

gboost7 = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gboost7.fit(X7_scaled, y)

y_pred7 = gboost6.predict(X7_scaled)
r2_7 = r2_score(y, y_pred7)

print(f"Gradient Boosting Model 7: R² = {r2_7:.3f}")

importances7 = pd.Series(gboost7.feature_importances_, index=X7.columns)
gboost_importances['Model 7'] = importances7
gboost_r2['Model 7'] = r2_6


In [ ]:
# Model 8
X8 = df_model_pca[['year', 'auction_house_dummy', 'online', 'desc_len'] + pca_features]
y = df_model_pca['log_realised_rmb']

scaler8 = StandardScaler()
X8_scaled = scaler8.fit_transform(X8)

gboost8 = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gboost8.fit(X8_scaled, y)

y_pred8 = gboost8.predict(X8_scaled)
r2_8 = r2_score(y, y_pred8)

print(f"Gradient Boosting Model 8: R² = {r2_8:.3f}")

importances8 = pd.Series(gboost8.feature_importances_, index=X8.columns)
gboost_importances['Model 8'] = importances8
gboost_r2['Model 8'] = r2_8

In [ ]:
# Model 9
X9 = df_model_themed[theme_features]
y = df_model_themed['log_realised_rmb']

scaler9 = StandardScaler()
X9_scaled = scaler9.fit_transform(X9)

gboost9 = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gboost9.fit(X9_scaled, y)

y_pred9 = gboost9.predict(X9_scaled)
r2_9 = r2_score(y, y_pred9)

print(f"Gradient Boosting Model 9: R² = {r2_9:.3f}")

importances9 = pd.Series(gboost9.feature_importances_, index=X9.columns)
gboost_importances['Model 9'] = importances9
gboost_r2['Model 9'] = r2_9

In [ ]:
# Model 10
X10 = df_model_themed[['year', 'auction_house_dummy', 'online', 'desc_len'] + theme_features]
y = df_model_themed['log_realised_rmb']

scaler10 = StandardScaler()
X10_scaled = scaler10.fit_transform(X9)

gboost10 = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gboost10.fit(X10_scaled, y)

y_pred10 = gboost10.predict(X10_scaled)
r2_10 = r2_score(y, y_pred10)

print(f"Gradient Boosting Model 10: R² = {r2_10:.3f}")

importances10 = pd.Series(gboost10.feature_importances_, index=X10.columns)
gboost_importances['Model 10'] = importances9
gboost_r2['Model 10'] = r2_9

In [ ]:
gboost_df = pd.DataFrame(gboost_importances)
gboost_df = gboost_df.round(3)
r2_series_gboost = pd.Series(gboost_r2, name='R²')
gboost_df_with_r2 = pd.concat([r2_series_gboost.to_frame().T, gboost_df])

In [ ]:
gboost_df_with_r2

Price to Estimate Ratio - Model 11

In [ ]:
df['log_ratio_realised_to_estimate'] = np.log(df['realised_rmb_adj'] / df['est_max_adj'])

In [ ]:
X = df[word_cols]
y = df['log_ratio_realised_to_estimate']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
X_train_sm = sm.add_constant(X_train)
model11 = sm.OLS(y_train, X_train_sm).fit()

staged_table = summary_col(
    results=[model1],
    model_names=['Model 11'],
    stars=True,
    float_format='%0.3f',
    info_dict={
        'R-squared': lambda x: f"{x.rsquared:.3f}",
        'N': lambda x: f"{int(x.nobs)}",
        'df_model': lambda x: f"{x.df_model:.0f}",
        'df_resid': lambda x: f"{x.df_resid:.0f}"
    }
)

staged_table

In [ ]:
lasso_coefs = pd.Series(lasso.coef_, index=X.columns)
lasso_r2 = r2_score(y_test, lasso.predict(X_test))

lasso_table = pd.DataFrame({
    'coef': lasso_coefs.round(3),
    'R-squared': [lasso_r2.round(3)] + [''] * (len(lasso_coefs) - 1)
})

lasso_table

In [ ]:
gbr_importances = pd.Series(gbr.feature_importances_, index=X.columns)
gbr_r2 = r2_score(y_test, gbr.predict(X_test))

gbr_table = pd.DataFrame({
    'importance': gbr_importances.round(3),
    'R-squared': [gbr_r2.round(3)] + [''] * (len(gbr_importances) - 1)
})

gbr_table